MACHINE TRANSLATION FOR KISWAHILI SAWA CORPUS USING JOEY NMT 

In [1]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "sw" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag


In [2]:
# Install opus-tools
! pip3 install opustools-pkg

The directory '/home/freshiasackey_gmail_com/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/freshiasackey_gmail_com/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 8.1.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
# Downloading JW300 corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-sw.xml.gz not found. The following files are available for downloading:

   8 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-sw.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  94 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/sw.zip

 365 MB Total size
./JW300_latest_xml_en-sw.xml.gz ... 100% of 8 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_sw.zip ... 100% of 94 MB


In [3]:
! pip3 install pandas
import pandas as pd

The directory '/home/freshiasackey_gmail_com/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/freshiasackey_gmail_com/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 8.1.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-06-02 19:47:29--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.002s  

2020-06-02 19:47:30 (108 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-06-02 19:47:30--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-sw.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Lengt

In [4]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


Loading JW + Sawa data to df

In [6]:
import pandas as pd
import csv

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
print(len(source))
print(len(target))

with open('sawa_corpus.csv', 'r', encoding = "ISO-8859-1") as readFile:

    reader = csv.DictReader(readFile)
    print(reader.fieldnames)
    for row in reader:
        source.append(row["'Source_language'"])
        target.append(row["'Target_language'"])        
print(len(source))
print(len(target))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)
print("Tail")
df.tail(10)

Loaded data and skipped 6478/979526 lines since contained in test set.
973049
973049
["'No'", "'Source_language'", "'Target_language'"]
0
1458206
1458206
cat

nyau

be foolish stupid

zuzuwaa

Dear Shareholders It is my pleasure to present to you the first Annual Report and Audited Financial Statements of AccessKenya Group following our successful listing on the Nairobi Stock Exchange in June 2007. 

WapendwaWenyehisa Nifuraha yangu kuu kuwasilisha kwenu taarifa ya kwanza ya kila mwaka na taarifa za ukaguzi wa hesabu za pesa za kundi la kampuni yaAccess Kenya kufuatia ufanisi wetu wa kuorodheshwa kwenye soko la hisa la Nairobi mwezi Juni mwaka wa 2007. 

Overriding all of these factors was an assessment of how effectively the target could roll out our new IT services solutions to the existingAccessKenya corporate customer base of approximately 2,000 customers.

Maswala yaliyopita haya yote ni ukadiriaji wa jinsi lengo hili litakavyoleta suluhisho kwa huduma zetu mpya za teknolojia ya h

,source_sentence,target_sentence
1458196,idiot\n,zuzu\n
1458197,blockhead\n,zuzu\n
1458198,simp-leton\n,zuzu\n
1458199,disturb\n,zuzua\n
1458200,derange\n,zuzua\n
1458201,infatuate\n,zuzua\n
1458202,craze\n,zuzua\n
1458203,vaunt fool flatter oneself\n,zuzuka\n
1458204,sink into oblivion\n,zuzuwaa\n
1458205,be foolish stupid\n,zuzuwaa\n


Pre-processing and export
It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [7]:
seed = 42  # Random seed for shuffling.
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/home/freshiasackey_gmail_com/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/freshiasackey_gmail_com/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [8]:
! pip3 install fuzzywuzzy

The directory '/home/freshiasackey_gmail_com/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/freshiasackey_gmail_com/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 8.1.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
! pip3 install python-Levenshtein

The directory '/home/freshiasackey_gmail_com/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/freshiasackey_gmail_com/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 8.1.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

00:00:00.35 0.00 percent complete
00:00:19.21 0.10 percent complete
00:00:37.93 0.21 percent complete
00:00:56.68 0.31 percent complete
00:01:15.59 0.41 percent complete
00:01:34.41 0.52 percent complete
00:01:52.94 0.62 percent complete
00:02:11.66 0.72 percent complete
00:02:30.61 0.83 percent complete
00:02:49.66 0.93 percent complete
00:03:08.77 1.03 percent complete
00:05:20.59 1.76 percent complete
00:05:39.32 1.86 percent complete


00:05:58.39 1.97 percent complete
00:06:18.16 2.07 percent complete
00:07:54.80 2.59 percent complete
00:08:13.84 2.69 percent complete
00:08:32.99 2.79 percent complete
00:08:51.73 2.90 percent complete


00:09:10.54 3.00 percent complete
00:09:29.24 3.10 percent complete
00:09:48.22 3.21 percent complete
00:10:07.34 3.31 percent complete
00:10:26.27 3.41 percent complete
00:10:45.62 3.52 percent complete
00:13:01.59 4.24 percent complete
00:13:20.79 4.34 percent complete
00:13:39.81 4.45 percent complete
00:13:59.35 4.55 percent complete
00:14:19.19 4.65 percent complete
00:14:38.59 4.76 percent complete
00:14:57.90 4.86 percent complete
00:15:17.13 4.97 percent complete


00:15:35.77 5.07 percent complete
00:15:54.30 5.17 percent complete
00:16:13.65 5.28 percent complete
00:16:33.36 5.38 percent complete
00:16:53.39 5.48 percent complete
00:17:12.03 5.59 percent complete
00:17:30.60 5.69 percent complete
00:17:49.93 5.79 percent complete
00:18:09.49 5.90 percent complete
00:18:28.06 6.00 percent complete
00:18:46.59 6.10 percent complete
00:19:05.22 6.21 percent complete
00:19:25.43 6.31 percent complete
00:19:44.72 6.41 percent complete
00:20:03.45 6.52 percent complete
00:20:22.44 6.62 percent complete
00:20:41.72 6.72 percent complete
00:21:00.81 6.83 percent complete
00:21:20.22 6.93 percent complete
00:21:38.89 7.03 percent complete
00:21:58.99 7.14 percent complete
00:22:17.81 7.24 percent complete
00:22:37.31 7.34 percent complete
00:22:56.68 7.45 percent complete
00:23:16.03 7.55 percent complete
00:23:35.40 7.65 percent complete
00:23:54.99 7.76 percent complete
00:24:16.28 7.86 percent complete
00:24:36.35 7.97 percent complete
00:24:55.59 8.

00:27:12.39 8.79 percent complete
00:27:31.93 8.90 percent complete
00:27:50.59 9.00 percent complete
00:28:09.81 9.10 percent complete
00:28:29.52 9.21 percent complete
00:28:48.78 9.31 percent complete
00:29:07.76 9.41 percent complete
00:29:27.28 9.52 percent complete
00:29:46.68 9.62 percent complete
00:30:06.83 9.72 percent complete
00:30:25.30 9.83 percent complete
00:30:44.66 9.93 percent complete
00:31:03.99 10.03 percent complete
00:31:24.08 10.14 percent complete
00:31:43.69 10.24 percent complete
00:32:02.83 10.34 percent complete
00:32:22.69 10.45 percent complete
00:32:41.88 10.55 percent complete
00:33:00.32 10.65 percent complete
00:33:19.15 10.76 percent complete
00:33:38.60 10.86 percent complete
00:33:57.41 10.97 percent complete
00:34:17.07 11.07 percent complete
00:34:36.60 11.17 percent complete
00:34:56.37 11.28 percent complete


...

04:33:02.88 88.76 percent complete
04:33:21.81 88.86 percent complete
04:33:41.06 88.96 percent complete
04:33:59.79 89.07 percent complete
04:34:19.57 89.17 percent complete
04:34:39.21 89.27 percent complete
04:34:58.19 89.38 percent complete
04:35:17.11 89.48 percent complete
04:35:35.72 89.58 percent complete
04:35:54.99 89.69 percent complete
04:36:13.80 89.79 percent complete
04:36:33.07 89.89 percent complete
04:36:52.15 90.00 percent complete
04:37:11.35 90.10 percent complete
04:37:30.43 90.20 percent complete
04:37:49.28 90.31 percent complete
04:38:08.53 90.41 percent complete


04:38:27.36 90.51 percent complete
04:38:46.17 90.62 percent complete
04:39:05.21 90.72 percent complete
04:39:24.27 90.82 percent complete
04:39:43.48 90.93 percent complete
04:40:02.25 91.03 percent complete
04:40:21.00 91.13 percent complete
04:40:39.78 91.24 percent complete
04:40:58.81 91.34 percent complete
04:41:17.12 91.44 percent complete
04:41:35.87 91.55 percent complete
04:41:54.83 91.65 percent complete
04:42:14.28 91.76 percent complete
04:42:33.48 91.86 percent complete
04:42:52.08 91.96 percent complete
04:43:10.91 92.07 percent complete
04:43:29.30 92.17 percent complete
04:43:48.27 92.27 percent complete
04:44:07.95 92.38 percent complete
04:44:26.65 92.48 percent complete
04:44:45.61 92.58 percent complete
04:45:04.44 92.69 percent complete
04:45:23.31 92.79 percent complete
04:45:42.86 92.89 percent complete
04:46:02.62 93.00 percent complete
04:46:21.53 93.10 percent complete
04:46:40.96 93.20 percent complete
04:46:59.97 93.31 percent complete
04:47:19.52 93.41 pe

05:04:45.58 99.10 percent complete
05:05:03.98 99.20 percent complete
05:05:23.30 99.31 percent complete
05:05:42.35 99.41 percent complete
05:06:01.88 99.51 percent complete


']


05:06:20.66 99.62 percent complete
05:06:39.71 99.72 percent complete
05:06:58.38 99.82 percent complete
05:07:17.83 99.93 percent complete


In [12]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")


# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
She gives birth to the twins Esau and Jacob .
Africa News of June 22 , 2006 , reporting on a “ workshop on sexual abuse and pornography ” in slums in one part of Uganda , said it is the “ parent’s neglect that has increased prostitution and drug abuse in the area . ”
Insight on the Scriptures says : “ Christians are not required to forgive those who practice malicious , willful sin with no repentance .
Now the effects of mechanical noise became apparent as factory workers suffered damage to their hearing .
Penne Restad , in her book Christmas in America — A History , says : “ Some clergy stressed that fallen humankind needed a season of abandon and excess , as long as it was carried on under the umbrella of Christian supervision . ”
We have seen that ‘ our labor is not in vain . ’
True , in ancient Israel some faithful individuals loved Jehovah’s law intensely .
162 ⇩
The French Manuel Biblique states : “ People knew that the seventy weeks of years fixed by Daniel were

Installation of JOEY NMT

In [13]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

fatal: destination path 'joeynmt' already exists and is not an empty directory.
The directory '/home/freshiasackey_gmail_com/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/freshiasackey_gmail_com/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
Processing /home/freshiasackey_gmail_com/sawajw/joeynmt


You are using pip version 8.1.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
! sudo pip3 install subword-nmt

The directory '/home/freshiasackey_gmail_com/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/freshiasackey_gmail_com/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 8.1.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Preprocessing the Data into Subword BPE Tokens
One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization (Sennrich, 2015) .

It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages (Sennrich, 2019) (Martinus, 2019)

Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by (Sennrich, 2019). You do not need to change anything. Simply running the below will be suitable.

In [25]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < testdata/test_custom.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < testdata/test_custom.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Swahili Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.sw     test.sw	   train.en
dev.bpe.en	dev.sw	     test.en	     train.bpe.en  train.sw
dev.bpe.sw	test.bpe.en  test.en-any.en  train.bpe.sw  vocab.txt
BPE Swahili Sentences

E P H : 6 : 2 2 a m b a y e n a m p e l e k a k w e n u k w a k u s u d i l i l o h i l o m p a t e k u z i j u a h a b a r i z e t u , n a y e a w a f a r i j i m i o y o y e n u .   

m u r u a 

Combined BPE Vocab
ḱ@@
о@@
о
м@@
є@@
м
ю
ж@@
Б@@
➊


Creating the JoeyNMT Config
JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

We used Transformer architecture
We set our dropout to reasonably high: 0.3 (recommended in (Sennrich, 2019))
Things worth playing with:

The batch size (also recommended to change for low-resourced languages)
The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
The decoder options (beam_size, alpha)
Evaluation metrics (BLEU versus Crhf4)

In [26]:
# More experiments to be done with the parameters

name = "ensw2"


# Create the config
config = """
name: "ensw2_transformer"
data:
    src: "en"
    trg: "sw"
    dev:   "data/ensw/dev.bpe"
    test:  "data/ensw/test.bpe"
    train:  "data/ensw/train.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/ensw/vocab.txt"
    trg_vocab: "data/ensw/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/ensw_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
"""

with open("joeynmt/configs/transformer_ensw_sawajw.yaml".format(name=name),'w') as f:
    f.write(config)

Train the Model
This single line of joeynmt runs the training using the config we made above

In [27]:
# Train the model 
!cd joeynmt;pwd; ls; python3 -m joeynmt train ./configs/transformer_ensw_sawajw.yaml;ls

/home/freshiasackey_gmail_com/sawajw/joeynmt
benchmarks.md	    data     joey-small.png  requirements.txt  test
CODE_OF_CONDUCT.md  docs     LICENSE	     scripts
configs		    joeynmt  README.md	     setup.py
/home/freshiasackey_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/freshiasackey_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/freshiasackey_gmail_com/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (t

2020-06-03 17:53:00,797 cfg.name                           : ensw2_transformer
2020-06-03 17:53:00,797 cfg.data.src                       : en
2020-06-03 17:53:00,797 cfg.data.trg                       : sw
2020-06-03 17:53:00,797 cfg.data.dev                       : data/ensw/dev.bpe
2020-06-03 17:53:00,797 cfg.data.test                      : data/ensw/test.bpe
2020-06-03 17:53:00,797 cfg.data.train                     : data/ensw/train.bpe
2020-06-03 17:53:00,797 cfg.data.level                     : bpe
2020-06-03 17:53:00,797 cfg.data.lowercase                 : False
2020-06-03 17:53:00,797 cfg.data.max_sent_length           : 100
2020-06-03 17:53:00,798 cfg.data.src_vocab                 : data/ensw/vocab.txt
2020-06-03 17:53:00,798 cfg.data.trg_vocab                 : data/ensw/vocab.txt
2020-06-03 17:53:00,798 cfg.testing.beam_size              : 5
2020-06-03 17:53:00,798 cfg.testing.alpha                  : 1.0
2020-06-03 17:53:00,798 cfg.training.random_seed           : 42
20

2020-06-03 17:56:31,636 Epoch   1 Step:     1100 Batch Loss:     4.322537 Tokens per Sec:    17309, Lr: 0.000300
2020-06-03 17:56:46,373 Epoch   1 Step:     1200 Batch Loss:     4.447544 Tokens per Sec:    17243, Lr: 0.000300
2020-06-03 17:57:01,408 Epoch   1 Step:     1300 Batch Loss:     4.431138 Tokens per Sec:    17227, Lr: 0.000300
2020-06-03 17:57:16,115 Epoch   1 Step:     1400 Batch Loss:     4.575477 Tokens per Sec:    17148, Lr: 0.000300
2020-06-03 17:57:30,974 Epoch   1 Step:     1500 Batch Loss:     4.376279 Tokens per Sec:    17173, Lr: 0.000300
2020-06-03 17:57:45,702 Epoch   1 Step:     1600 Batch Loss:     4.329970 Tokens per Sec:    17201, Lr: 0.000300
2020-06-03 17:58:00,429 Epoch   1 Step:     1700 Batch Loss:     4.079594 Tokens per Sec:    17188, Lr: 0.000300
2020-06-03 17:58:15,046 Epoch   1 Step:     1800 Batch Loss:     4.365270 Tokens per Sec:    17022, Lr: 0.000300
2020-06-03 17:58:29,967 Epoch   1 Step:     1900 Batch Loss:     3.984522 Tokens per Sec:    169

2020-06-03 18:05:48,433 Epoch   1 Step:     4100 Batch Loss:     3.317481 Tokens per Sec:    17435, Lr: 0.000300
2020-06-03 18:06:03,372 Epoch   1 Step:     4200 Batch Loss:     3.246864 Tokens per Sec:    17137, Lr: 0.000300
2020-06-03 18:06:18,054 Epoch   1 Step:     4300 Batch Loss:     3.293374 Tokens per Sec:    17207, Lr: 0.000300
2020-06-03 18:06:32,941 Epoch   1 Step:     4400 Batch Loss:     3.260285 Tokens per Sec:    17275, Lr: 0.000300
2020-06-03 18:06:47,798 Epoch   1 Step:     4500 Batch Loss:     3.459508 Tokens per Sec:    17502, Lr: 0.000300
2020-06-03 18:07:02,502 Epoch   1 Step:     4600 Batch Loss:     3.344480 Tokens per Sec:    17039, Lr: 0.000300
2020-06-03 18:07:17,328 Epoch   1 Step:     4700 Batch Loss:     3.113860 Tokens per Sec:    17135, Lr: 0.000300
2020-06-03 18:07:32,065 Epoch   1 Step:     4800 Batch Loss:     3.096684 Tokens per Sec:    17125, Lr: 0.000300
2020-06-03 18:07:46,990 Epoch   1 Step:     4900 Batch Loss:     2.981090 Tokens per Sec:    172

2020-06-03 18:14:17,493 	Reference:  Mara nyingi kichocheo kingine ni tamaa ya kuhubiri katika eneo lenye matokeo zaidi — nacho chafaa .
2020-06-03 18:14:17,494 	Hypothesis: Mara nyingi , tamaa ya kufanya kazi katika majumba ya madogo ni yenye kupendeza — na kwa sababu ya kukosa .
2020-06-03 18:14:17,494 Validation result (greedy) at epoch   1, step     7000: bleu:  11.49, loss: 73131.0000, ppl:  13.8836, duration: 26.0411s
2020-06-03 18:14:32,457 Epoch   1 Step:     7100 Batch Loss:     3.107884 Tokens per Sec:    17404, Lr: 0.000300
2020-06-03 18:14:47,394 Epoch   1 Step:     7200 Batch Loss:     3.149327 Tokens per Sec:    17124, Lr: 0.000300
2020-06-03 18:15:02,001 Epoch   1 Step:     7300 Batch Loss:     2.778289 Tokens per Sec:    17058, Lr: 0.000300
2020-06-03 18:15:16,989 Epoch   1 Step:     7400 Batch Loss:     2.620528 Tokens per Sec:    17040, Lr: 0.000300
2020-06-03 18:15:32,124 Epoch   1 Step:     7500 Batch Loss:     2.898374 Tokens per Sec:    17213, Lr: 0.000300
2020-06

...

2020-06-04 07:54:21,191 Epoch  29 Step:   294100 Batch Loss:     1.559919 Tokens per Sec:    17434, Lr: 0.000072
2020-06-04 07:54:36,129 Epoch  29 Step:   294200 Batch Loss:     1.356738 Tokens per Sec:    17579, Lr: 0.000072
2020-06-04 07:54:50,739 Epoch  29 Step:   294300 Batch Loss:     1.781323 Tokens per Sec:    17303, Lr: 0.000072
2020-06-04 07:55:05,562 Epoch  29 Step:   294400 Batch Loss:     1.420566 Tokens per Sec:    17363, Lr: 0.000072
2020-06-04 07:55:20,348 Epoch  29 Step:   294500 Batch Loss:     1.454908 Tokens per Sec:    17162, Lr: 0.000072
2020-06-04 07:55:35,225 Epoch  29 Step:   294600 Batch Loss:     1.295670 Tokens per Sec:    17082, Lr: 0.000072
2020-06-04 07:55:50,066 Epoch  29 Step:   294700 Batch Loss:     1.417656 Tokens per Sec:    17138, Lr: 0.000072
2020-06-04 07:56:04,914 Epoch  29 Step:   294800 Batch Loss:     1.344323 Tokens per Sec:    17119, Lr: 0.000072
2020-06-04 07:56:19,559 Epoch  29 Step:   294900 Batch Loss:     1.595331 Tokens per Sec:    172

2020-06-04 08:02:50,579 Epoch  29: total training loss 14585.55
2020-06-04 08:02:50,579 EPOCH 30
2020-06-04 08:02:56,300 Epoch  30 Step:   297100 Batch Loss:     1.381225 Tokens per Sec:    12962, Lr: 0.000072
2020-06-04 08:03:11,091 Epoch  30 Step:   297200 Batch Loss:     1.397844 Tokens per Sec:    17290, Lr: 0.000072
2020-06-04 08:03:25,969 Epoch  30 Step:   297300 Batch Loss:     1.424298 Tokens per Sec:    17194, Lr: 0.000072
2020-06-04 08:03:40,794 Epoch  30 Step:   297400 Batch Loss:     1.388924 Tokens per Sec:    17191, Lr: 0.000072
2020-06-04 08:03:55,591 Epoch  30 Step:   297500 Batch Loss:     1.426854 Tokens per Sec:    17602, Lr: 0.000072
2020-06-04 08:04:10,375 Epoch  30 Step:   297600 Batch Loss:     1.912871 Tokens per Sec:    17158, Lr: 0.000072
2020-06-04 08:04:25,165 Epoch  30 Step:   297700 Batch Loss:     1.343861 Tokens per Sec:    17185, Lr: 0.000072
2020-06-04 08:04:39,832 Epoch  30 Step:   297800 Batch Loss:     1.558529 Tokens per Sec:    17278, Lr: 0.000072

2020-06-04 08:11:29,494 Epoch  30 Step:   300100 Batch Loss:     1.432807 Tokens per Sec:    17315, Lr: 0.000072
2020-06-04 08:11:44,276 Epoch  30 Step:   300200 Batch Loss:     1.392327 Tokens per Sec:    17170, Lr: 0.000072
2020-06-04 08:11:59,172 Epoch  30 Step:   300300 Batch Loss:     1.476485 Tokens per Sec:    17287, Lr: 0.000072
2020-06-04 08:12:14,008 Epoch  30 Step:   300400 Batch Loss:     1.510286 Tokens per Sec:    17032, Lr: 0.000072
2020-06-04 08:12:28,919 Epoch  30 Step:   300500 Batch Loss:     1.563306 Tokens per Sec:    17561, Lr: 0.000072
2020-06-04 08:12:43,848 Epoch  30 Step:   300600 Batch Loss:     1.366653 Tokens per Sec:    17083, Lr: 0.000072
2020-06-04 08:12:58,781 Epoch  30 Step:   300700 Batch Loss:     1.378415 Tokens per Sec:    17140, Lr: 0.000072
2020-06-04 08:13:13,465 Epoch  30 Step:   300800 Batch Loss:     1.217507 Tokens per Sec:    17155, Lr: 0.000072
2020-06-04 08:13:28,082 Epoch  30 Step:   300900 Batch Loss:     1.776078 Tokens per Sec:    169

2020-06-04 08:20:16,495 Epoch  30 Step:   303200 Batch Loss:     1.413313 Tokens per Sec:    17098, Lr: 0.000050
2020-06-04 08:20:30,991 Epoch  30 Step:   303300 Batch Loss:     1.435676 Tokens per Sec:    16852, Lr: 0.000050
2020-06-04 08:20:45,986 Epoch  30 Step:   303400 Batch Loss:     1.425150 Tokens per Sec:    17555, Lr: 0.000050
2020-06-04 08:21:00,686 Epoch  30 Step:   303500 Batch Loss:     1.488713 Tokens per Sec:    17589, Lr: 0.000050
2020-06-04 08:21:15,527 Epoch  30 Step:   303600 Batch Loss:     1.349584 Tokens per Sec:    17256, Lr: 0.000050
2020-06-04 08:21:30,435 Epoch  30 Step:   303700 Batch Loss:     1.642421 Tokens per Sec:    17249, Lr: 0.000050
2020-06-04 08:21:45,411 Epoch  30 Step:   303800 Batch Loss:     1.413663 Tokens per Sec:    17148, Lr: 0.000050
2020-06-04 08:22:00,133 Epoch  30 Step:   303900 Batch Loss:     1.474272 Tokens per Sec:    17388, Lr: 0.000050
2020-06-04 08:22:14,900 Epoch  30 Step:   304000 Batch Loss:     1.384291 Tokens per Sec:    171

2020-06-04 08:28:33,677 Epoch  30 Step:   306100 Batch Loss:     1.394271 Tokens per Sec:    17254, Lr: 0.000050
2020-06-04 08:28:48,464 Epoch  30 Step:   306200 Batch Loss:     1.633674 Tokens per Sec:    17083, Lr: 0.000050
2020-06-04 08:29:03,325 Epoch  30 Step:   306300 Batch Loss:     1.349516 Tokens per Sec:    17222, Lr: 0.000050
2020-06-04 08:29:18,098 Epoch  30 Step:   306400 Batch Loss:     1.380888 Tokens per Sec:    16933, Lr: 0.000050
2020-06-04 08:29:32,760 Epoch  30 Step:   306500 Batch Loss:     1.501461 Tokens per Sec:    17236, Lr: 0.000050
2020-06-04 08:29:47,493 Epoch  30 Step:   306600 Batch Loss:     1.508789 Tokens per Sec:    17340, Lr: 0.000050
2020-06-04 08:30:02,258 Epoch  30 Step:   306700 Batch Loss:     1.248501 Tokens per Sec:    17192, Lr: 0.000050
2020-06-04 08:30:17,157 Epoch  30 Step:   306800 Batch Loss:     1.424952 Tokens per Sec:    17046, Lr: 0.000050
2020-06-04 08:30:32,057 Epoch  30 Step:   306900 Batch Loss:     1.521583 Tokens per Sec:    170